# Assignment 3 part 2 

## Task HPV slides

Using the HPV dataset I want you to design a model that can detect if a sample is from a tumor or not. 

Choose a neural network you think will work and try to get above 80% accuracy

Choose a neural network you do not think will work, run it atleast once. 

None of the neworks can be a Multi Layer Perceptron.

There's a whole thesis written based on this dataset, available here: http://uu.diva-portal.org/smash/get/diva2:1650957/FULLTEXT01.pdf . The thesis then resulted in a paper: https://openaccess.thecvf.com/content_ICCV_2017_workshops/w1/html/Wieslander_Deep_Convolutional_Neural_ICCV_2017_paper.html 

**You are not allowed to use the exact same network and configurations as in these texts, or as in your hand in for the mandatory assignment, but you are allowed to use the same base network if you are using transfer learning.**  

You may discuss theory with other groups, but not code nor share code. You may not use ChatGPT4 or similar programs to generate a solution or code.

 <span style="color:red"> Solutions deemed to similar will be run through a program designed to detect coding plagirism and if flagged you will be reported for plagirism. </span> 
 
 One hand in by the group.

## Hand-in

To get the bonus point you need to turn in a runable notebook that contains:
1. A 1-5 scentence motivation of why you chose your networks with references to sources describing the network type.  

2. a) A neural network that gets atleast an 80% accuracy   
   
    b) If you do not reach this whithin a reasonable time frame; the network you have and a well motivated analysis of what could be done differently to improve the accuracy of your model. (Atleast 3 improvement strategies, 1-5 paragraphs) These cannot be the same strategies you used in the mandatory assignment. 


3. A badly performing network run atleast once. Motivate why this network was not as good a choice as the one you choose. Include references to the source of the network if you're using a pre existing network (1 paragraph)

The first, good neural network is the focus of this bonus assignment and you will choose how to best allocate your time

For the bad network I wouldn't recommend spending more than an hour on this unless you really, really want.

4. Evaluate your best network using atleast 3 metrics and discuss their differences/similarities in relation to your results.  (1-3 paragraphs)

Optional: Make a leave-one-out analysis of your best network to show that you weren't simply lucky in how you divided your data set. 

## Bonus points
1 bonus point: 1, 2 a *or* b, and 3 all answered correctly 
1 bonus point: 4 answered and discussed

So for 2 bonus points: 1, 2a *or* 2b, 3, 4 all answered correctly 

## Q1. A 1-5 scentence motivation of why you chose your networks with references to sources describing the network type.  


# T1. A neural network that gets atleast an 80% accuracy

In [11]:
## IMPORTS
import numpy as np
import tensorflow as tf
import pandas as pd
import IPython
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from datetime import datetime
import cv2
import os
from PIL import Image

#VGG16
from keras.applications.vgg16 import VGG16

#ResNet
from tensorflow.keras.applications import ResNet50

#EfficientNet
from tensorflow.keras.applications import EfficientNetB7 as EfficientNet

#Lab scripts
import plot_helper
import cnn_helper



## DATA PREPARATION

## Directory
base_directory = "/home/jovyan/big-data_deeplearning/"
images_path = base_directory + "LabData/cell_images_small/"

## Create a Dataframe for each set of images, adding a category column with H for healthy and T for tumor

# training dataset
for folder in ["test", "train", "validation"]:
    data = []
    for category in ["healthy", "tumor"]:
        category_path = os.path.join(images_path, folder, category)
        for filename in os.listdir(category_path):
            imageID = filename
            classification = "H" if category == "healthy" else "T"
            data.append({"imageID" : imageID, "classification" : classification})
    if folder == "test":
        df_test = pd.DataFrame(data)
        # see the weights of each class
        print("Test")
        print(df_test['classification'].value_counts())
    if folder == "train":
        df_train = pd.DataFrame(data)
        # see the weights of each class
        print("Train")
        print(df_train['classification'].value_counts())
    if folder == "validation":
        df_validation = pd.DataFrame(data)
        # see the weights of each class
        print("Validation")
        print(df_validation['classification'].value_counts())
            

## Check image shape
photo = Image.open("/home/jovyan/big-data_deeplearning/LabData/cell_images_small/test/healthy/cell_10088.tif")
array_photo = np.array(photo)
print("Shape of the images is: ", array_photo.shape)
image_shape = array_photo.shape


Test
H    1572
T     746
Name: classification, dtype: int64
Train
H    5153
T    2341
Name: classification, dtype: int64
Validation
H    1270
T     588
Name: classification, dtype: int64
Shape of the images is:  (48, 48)


In [5]:
## DATA GENERATOR

## Directory with all the images:
train_path = "/home/jovyan/big-data_deeplearning/LabData/cell_images_small/train/all/"
test_path = "/home/jovyan/big-data_deeplearning/LabData/cell_images_small/test/all/"
validation_path = "/home/jovyan/big-data_deeplearning/LabData/cell_images_small/validation/all/"

## Set up generators for all datasets
image_shape = (48, 48, 3)

batch_size = 32
filename_column = "imageID"
true_label_column = "classification"

## Doing the preprocessing required
train_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


train_generator = train_data_generator.flow_from_dataframe(
    df_train, directory=train_path, x_col=filename_column, y_col=true_label_column,
    class_mode='binary', batch_size=batch_size, target_size = image_shape[:2], color_mode='rgb', shuffle=False)

valid_generator = valid_data_generator.flow_from_dataframe(
    df_validation, directory=validation_path, x_col=filename_column, y_col=true_label_column,
    class_mode='binary', batch_size=batch_size, target_size = image_shape[:2], color_mode='rgb', shuffle=False)


test_generator = test_data_generator.flow_from_dataframe(
    df_test, directory=test_path, x_col=filename_column, y_col=true_label_column,
    class_mode='binary', batch_size=batch_size, target_size = image_shape[:2], color_mode='rgb', shuffle=False)


train_steps = train_generator.n//train_generator.batch_size if train_generator.n >= train_generator.batch_size else 1
validation_steps = valid_generator.n//valid_generator.batch_size if valid_generator.n >= valid_generator.batch_size else 1


/usr/local/lib/python3.8/dist-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="imageID". These filename(s) will be ignored.
  warnings.warn(


Found 7493 validated image filenames belonging to 2 classes.
Found 1858 validated image filenames belonging to 2 classes.
Found 2317 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.8/dist-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="imageID". These filename(s) will be ignored.
  warnings.warn(


In [6]:
## Instantiate the base model EfficientNet
EfficientNet_base_model = EfficientNet(weights = 'imagenet', input_shape = image_shape, include_top = False) 

## Freeze the base model, so that we don't modify the weights when backpropagating
EfficientNet_base_model.trainable = False

# Our top layers, and the model that we will train.
last_layer = EfficientNet_base_model.output
x = keras.layers.Flatten()(last_layer) # could be GlobalAveragePooling2D, GlobalMaxPooling2D or Flatten as well, not sure how to pick
x = keras.layers.Dense(512, activation = 'relu')(x)
x = keras.layers.Dropout(0.5)(x)  # To reduce overfittin, since in the plots we are getting a lot.
#x = keras.layers.Dense(512, activation = 'relu')(x)
x = keras.layers.Dense(2, activation = 'sigmoid')(x)

# Ensamble the new full network
EfficientNet_model = tf.keras.Model(EfficientNet_base_model.input, x, name = "EfficientNet_model")

#EfficientNet_model.summary()

## Compile the new model, using the base model + the new layers that we added
EfficientNet_model.compile(optimizer = keras.optimizers.Adam(), 
                    loss = keras.losses.BinaryCrossentropy(),
                    metrics = [keras.metrics.Accuracy()])

## Actually train model
epochs = 10
EfficientNet_history = EfficientNet_model.fit(train_generator,
                    steps_per_epoch = train_steps,
                    validation_data = valid_generator,
                    validation_steps = validation_steps,
                    epochs = epochs,
                    batch_size = batch_size,
                    verbose = 1)


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/lib/python3.8/d

## "I have some questions about the instructions"

Once you've started to ask yourself questions about the task, read the below and if questions remain ask any TA or me during any lesson or lab. 

**Wait, where's the code?** ***Nowhere*** , you'll design this solution all on your own ***as per feedback from last years course evaluation. Your answers on the course evaluation will be incorporated into next years teaching, so do fill out the course evaluation***. Do make sure your solution is clear and be extremely clear and indicate when you're answering questions. Make a new notebook. 

**The thesis is 80 pages long. How are we supposed to read this *and* do the assignment?** I do not expect you to read the full thesis, the article on the other hand is 8 pages which you could atleast skim.

**How do I make a new notebook?** Ask any TA or lecturer. If it turns out to be very difficult for a student to generate a notebook in the environment just continue below the answers here. Write a solution header to indicate the start of your code. 


**What is a reasonable time frame?**: You decide. I'd say atleast 4 hours to conclude part 2 successfully, around 1 hour for part 3, but it is your education and you decide how much time to spend on this part. This is for 1 bonus point and more of a challenge than the other assignment. I do not think most groups will be able to make a network that reaches 88%, depending on the classes they choose, nor do I expect all groups to attempt this bonus part. 

**Can we work with other groups?**: You may discuss theory, but not code nor share code. Solutions deemed to similar will be run through a program designed to detect coding plagirism and if flagged you will be reported for plagirism. 

**This is so much fun, can we turn in more than 2 networks?**: Turn in one notebook containing only what is asked for and I will grade this. If time allows (and I do find this fun so it's very likely that time will allow) I will give you feedback on your other choices as well. While limiting your answers to only what is asked for is good practice for the exam, if you want to explore things further in a seperate notebook feel free. 


(*If you're reading this and wondering why I think this question and clarification is needed it's because I've (happily) run into this in the past. I appreciate that some people enjoy their education to the point of exploring beyond the assignments, and I understand that some people have too many things to get done to do everything fun. So you do you.*)

**We've tried so hard and we can't get a good result!:** I'm guessing you've spent a fair amount of time on this. Write what you think would work if you had more time, and if you've tried all strategies you can think of pick 3 and describe them and hand that in with the network. 


**We are split, some of us want to give up, but some don't** Have you spent atleast 2.5 hours on task 2a? Then you can consider yourself done with that part. Move on to 2b. 

**We don't want to give up!** If you feel like moving on to part 2b is giving up, think of it more like practicing for a job: Some employers will just want you to get things done, some employers want you to spend just a bit of time on something and if it turns out to be a much bigger task than expected then the resource (your time) needed to get it done might not be worth the possible gain of getting it done.  

Turning in possible solution strategies might be appreciated. Managing your time is an important life skill. (Do not ask any phd-student for advice on this though and expect us to do what we say, we're not always good role models.)

**How long is a paragraph?** Preferably around this answers lenght. Punctuation is appreciated, here and in the exam. I've tried to write 1-5 scentences to indicate lenght, but I'd rather have many short sentences than 1 long one. If your scentence is more than 2 lines it needs a punctuation. *(Do as I say, not as I do. I do try to do as I say)*

**We got more than 88%! Do we have to write extra improvement strategies?**: Good job! Absolutely not. If you really, *really*, want feedback on them then include them. 

**Do we have to be able to do this assignment to pass the exam?** This assignment builds on theory from the lectures, and I do expect you to understand the theory for the exam. However, this part is purposefully a more difficult assignment and your ability to complete this will not determine wheter or not you'll pass the deep learning part of the exam.

**These runs take forever!** I'd like to remind you that my runs take up to 60 hours. However when I'm prototyping I'm not running the full k-folds I do when I properly test the model, so it only takes around 20 minutes. Is there anything you can do to speed up your runs? While the program runs find something else to do, like planning the rest of your solution or reviewing what you've learned in deep learning so far.  